In [1]:
!pip install ofa
!pip install torch==1.4.0 torchvision==0.5.0

     |████████████████████████████████| 106 kB 7.8 MB/s 
     |████████████████████████████████| 753.4 MB 6.0 kB/s 
     |████████████████████████████████| 4.0 MB 52.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.


In [4]:
import copy

import torch

from ofa.model_zoo import ofa_net

raw_resnet = ofa_net('ofa_resnet50', pretrained=True)
raw_resnet.set_max_net()
max_subnet = raw_resnet.get_active_subnet()
raw_resnet.set_active_subnet(d=0, w=0)
min_subnet = raw_resnet.get_active_subnet()

Downloading: "https://hanlab.mit.edu/files/OnceForAll/ofa_nets/ofa_resnet50_d=0+1+2_e=0.2+0.25+0.35_w=0.65+0.8+1.0" to .torch/ofa_nets/ofa_resnet50_d=0+1+2_e=0.2+0.25+0.35_w=0.65+0.8+1.0


In [5]:
max_subnet.eval()
max_subnet.qconfig = torch.quantization.default_qconfig
max_subnet_1 = torch.quantization.prepare(max_subnet)

In [6]:
min_subnet.eval()
min_subnet.qconfig = torch.quantization.default_qconfig
min_subnet_1 = torch.quantization.prepare(min_subnet)

In [7]:
max_q1 = torch.quantization.convert(max_subnet_1, inplace=True)
min_q1 = torch.quantization.convert(min_subnet_1, inplace=True)

/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:172: UserWarning: Must run observer before calling calculate_qparams.                           Returning default scale and zero point.
  Returning default scale and zero point.")


In [25]:
max_mods = dict(max_q1.named_children())
print(max_mods['input_stem'])
print(len(max_mods['blocks']))
min_mods = dict(min_q1.named_children())
print(min_mods['input_stem'])
print(min_mods['blocks'][0])
print(len(min_mods['blocks']))


# for n in max_q1.named_buffers():
#     print(n)

ModuleList(
  (0): ConvLayer(
    (conv): QuantizedConv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), scale=1.0, zero_point=0, padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): QuantizedReLU(inplace=True)
  )
  (1): ResidualBlock(
    (conv): ConvLayer(
      (conv): QuantizedConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): QuantizedReLU(inplace=True)
    )
    (shortcut): IdentityLayer()
  )
  (2): ConvLayer(
    (conv): QuantizedConv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): QuantizedReLU(inplace=True)
  )
)
18
ModuleList(
  (0): ConvLayer(
    (conv): QuantizedConv2d(3, 24, kernel_size=(3, 3), str